<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_residual_blocks_m0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook we shall use some custom python modules where we have defined some custom layers to implement residual blocks.

The architecture here is a more or less direct rephrasing of the architecture in the repo called L2-vs-L1-vs-CB-loss-in-deep-CNN-based-AE-on-cifar10
in terms of residual blocks.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

import Residual_blocks as resblock

Cloning into 'AE-w.-Bottleneck-Residual-Blocks'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 166 (delta 81), reused 76 (delta 25), pack-reused 0
Receiving objects: 100% (166/166), 4.17 MiB | 8.58 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/AE-w.-Bottleneck-Residual-Blocks


# **Dataset (cifar10) loading**

In [3]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=123)

In [4]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteKIDSK1/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteKIDSK1/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


# **The model**

In [5]:
encoder_input = keras.Input(shape=input_size)

x = resblock.bridge_residual_conv2D_block(64, 1, 3, initializer, 'min', use_bn = 'False')(encoder_input)

x = resblock.residual_conv2D_block(64, 1, 2, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

#####################################################################################

# the default padding is 'valid'

x = resblock.bridge_residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.Flatten()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)


z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 bridge_residual_conv2d_blo  (None, 30, 30, 64)        2304      
 ck (bridge_residual_conv2D                                      
 _block)                                                         
                                                                 
 residual_conv2d_block (res  (None, 29, 29, 64)        16704     
 idual_conv2D_block)                                             
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 bridge_residual_conv2d_blo  (None, 12, 12, 128)       1073

In [6]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((2,2,1024))(x)

x = resblock.bridge_residual_conv2Dtrans_block(1024, 1, 1, initializer, use_bn = 'False')(x)

##############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(512, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(512, 2, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(256, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 1, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(64, 2, 2, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_30 (ReLU)             (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 768)               393984    
                                                                 
 re_lu_31 (ReLU)             (None, 768)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              787456    
                                                                 
 re_lu_32 (ReLU)             (None, 1024)              0   

In [7]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               7422144   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         12404995  
                                                                 
Total params: 19827139 (75.63 MB)
Trainable params: 19812931 (75.58 MB)
Non-trainable params: 14208 (55.50 KB)
_________________________________________________________________


# **Training**

In [ ]:
K.clear_session()

In [8]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=4, verbose=1, start_from_epoch=10, restore_best_weights=True)
optim = keras.optimizers.Adam(learning_rate = 0.0001)
AE.compile(optimizer=optim,loss='mse', metrics=['accuracy'])
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75, callbacks=[callback])

Epoch 1/75
391/391 [==============================] - 68s 99ms/step - loss: 0.0457 - accuracy: 0.4500 - val_loss: 0.0377 - val_accuracy: 0.4554
Epoch 2/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0300 - accuracy: 0.4675 - val_loss: 0.0308 - val_accuracy: 0.4675
Epoch 3/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0265 - accuracy: 0.4801 - val_loss: 0.0268 - val_accuracy: 0.4813
Epoch 4/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0250 - accuracy: 0.4983 - val_loss: 0.0250 - val_accuracy: 0.5081
Epoch 5/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0230 - accuracy: 0.5466 - val_loss: 0.0236 - val_accuracy: 0.5695
Epoch 6/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0210 - accuracy: 0.5697 - val_loss: 0.0222 - val_accuracy: 0.5829
Epoch 7/75
391/391 [==============================] - 34s 86ms/step - loss: 0.0200 - accuracy: 0.5787 - val_loss: 0.0205 - val_accuracy:

In [9]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=4, verbose=1, start_from_epoch=1, restore_best_weights=True)
optim = keras.optimizers.Adam(learning_rate = 0.00001)
AE.compile(optimizer=optim,loss='mse', metrics=['accuracy'])
history2 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75, callbacks=[callback])

Epoch 1/75
391/391 [==============================] - 54s 89ms/step - loss: 0.0102 - accuracy: 0.6757 - val_loss: 0.0129 - val_accuracy: 0.6597
Epoch 2/75
391/391 [==============================] - 33s 85ms/step - loss: 0.0101 - accuracy: 0.6763 - val_loss: 0.0130 - val_accuracy: 0.6614
Epoch 3/75
391/391 [==============================] - 33s 84ms/step - loss: 0.0100 - accuracy: 0.6765 - val_loss: 0.0130 - val_accuracy: 0.6616
Epoch 4/75
391/391 [==============================] - 33s 84ms/step - loss: 0.0100 - accuracy: 0.6770 - val_loss: 0.0130 - val_accuracy: 0.6597
Epoch 5/75
391/391 [==============================] - 33s 84ms/step - loss: 0.0100 - accuracy: 0.6766 - val_loss: 0.0130 - val_accuracy: 0.6599
Epoch 6/75
391/391 [==============================] - 33s 84ms/step - loss: 0.0100 - accuracy: 0.6777 - val_loss: 0.0130 - val_accuracy: 0.6602
Epoch 7/75
391/391 [==============================] - 33s 84ms/step - loss: 0.0099 - accuracy: 0.6777 - val_loss: 0.0130 - val_accuracy:

The accuracy obtained is poor in comparison with the other architectures.